In [1]:
import os
import sys
sys.path.append('..')
from data_center.tej_handler import TEJHandler
from monitor.tw_monitor import TWMarketMonitor

from pymongo import MongoClient
import tejapi
TEJHandler() # init handler inf

import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta

### 目前月營收、財報時間資料都有誤

In [2]:
mongo = MongoClient(TWMarketMonitor().client_url)
db = mongo['PSTAGE']
collection = db['financial_report']
df = pd.DataFrame(list(collection.find().sort('財務資料日', -1).limit(1000000)))
df

,_id,證券碼,財務資料日,季別,合併(Y/N),單季(Q)/單半年(H),月份,幣別,財報發布日,每股盈餘,...,營業利益,營業外收入及支出,所得稅費用,合併總損益,歸屬母公司淨利（損）,折舊－CFO,攤提－CFO,來自營運之現金流量,投資活動之現金流量,籌資活動之現金流量
0,6604db8aec0eccbba86526d5,000930,2023-12-01,4,Y,Q,12,NTD,2024-03-11,2.04,...,318776.0,NaN,74438.0,331687.0,331659.0,47824.0,22265.0,-728861.0,-49821.0,651527.0
1,6604db8aec0eccbba86526d8,000960,2023-12-01,4,Y,Q,12,NTD,2024-03-15,3.84,...,734790.0,NaN,210149.0,1564768.0,1564801.0,111534.0,48569.0,-5415113.0,-214243.0,823941.0
2,6604db8aec0eccbba86526db,0009A0,2023-12-01,4,Y,Q,12,NTD,2024-03-14,1.92,...,549008.0,NaN,130525.0,779095.0,779095.0,111403.0,33149.0,-11378205.0,2634040.0,7753619.0
3,6604db8aec0eccbba86526de,1101,2023-12-01,4,Y,Q,12,NTD,2024-02-28,1.00,...,3433504.0,383504.0,1317094.0,2499914.0,1801908.0,2082429.0,282945.0,13796962.0,-11346930.0,-5820763.0
4,6604db8aec0eccbba86526e1,1102,2023-12-01,4,Y,Q,12,NTD,2024-03-15,1.60,...,1467155.0,481242.0,667356.0,1281041.0,1325636.0,1124744.0,80517.0,3335886.0,-7680917.0,693873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108160,660678e360b8c294fcf82cf9,8712,2005-06-01,2,N,H,6,NTD,NaT,-2.70,...,-147867.0,-2581873.0,0.0,-2729740.0,-2729740.0,10228.0,3389.0,-31897.0,323676.0,-287406.0
108161,660678e360b8c294fcf82cfa,8725,2005-06-01,2,N,H,6,NTD,NaT,-3.65,...,-7160.0,-90791.0,0.0,-423249.0,-423249.0,158.0,0.0,18339.0,-1529.0,-37528.0
108162,660678e360b8c294fcf82d02,8913,2005-06-01,2,Y,H,6,NTD,NaT,-0.03,...,-15436.0,28417.0,1944.0,11037.0,11037.0,60924.0,36.0,128306.0,133327.0,-271975.0
108163,660678e360b8c294fcf82db6,8925,2005-06-01,2,N,H,6,NTD,NaT,0.19,...,41628.0,-14578.0,5930.0,21120.0,21120.0,21576.0,5024.0,-21634.0,-28630.0,-51783.0


In [6]:
org_fr_df = pd.read_table("/Volumes/Database/Original_data/IFRS合併一般產業單季/財報.txt", encoding='Big5-HKSCS', low_memory=False)
org_fr_df

form_new =  df[['CFO/負債','CFO/合併總損益']]
old_fr_data1 = org_fr_df [['公司',
                    '年/月', 
                    '季別', 
                    '合併(Y/N)',
                    '單季(Q)/單半年(H)',
                    '月份',
                    '幣別',
                    '財報發布日',
                    '每股盈餘',
                    'ROA(C)稅前息前折舊前',
                    'ROE(A)－稅後',
                    '營業毛利率',
                    '營業利益率',
                    '稅後淨利率']]

old_fr_data2 = org_fr_df [[
                    '現金流量比率',
                    '每股淨值(F)－TSE公告數',
                    '營收成長率',
                    '營業毛利成長率',
                    '營業利益成長率',
                    '流動比率',
                    '速動比率',
                    '負債比率',
                    '長期資金適合率(A)',
                    '應收帳款週轉次數',
                    '總資產週轉次數',
                    '存貨週轉率（次）',
                    '  應收帳款及票據',
                    '  存貨',
                    '流動資產',
                    '  採權益法之長期股權投資',
                    '  不動產廠房及設備',
                    '  商譽及無形資產合計',
                    '非流動資產',
                    '資產總額',
                    '  短期借款',
                    '  應付商業本票／承兌匯票',
                    '  一年內到期長期負債',
                    '流動負債',
                    '  應付公司債－非流動',
                    '  銀行借款－非流動',
                    '  其他長期借款－非流動',
                    '非流動負債',
                    '負債總額',
                    '  股本',
                    '  保留盈餘',
                    '股東權益總額',
                    '負債及股東權益總額',
                    '營業收入淨額',
                    '營業成本',
                    '營業毛利',
                    '營業費用',
                    '營業利益',
                    '營業外收入及支出',
                    '所得稅費用',
                    '合併總損益',
                    '歸屬母公司淨利（損）',
                    '  折舊－CFO',
                    '  攤提－CFO',
                    '來自營運之現金流量',
                    '投資活動之現金流量',
                    '籌資活動之現金流量']
] 

In [ ]:
data = pd.concat([form_new, old_fr_data2], axis = 1)
data = pd.concat([old_fr_data1, data], axis = 1)
data.columns = df.columns[1:] 
data

In [3]:
data_code = "TWN/EWIFINQ"
pdata = TWMarketMonitor().pdata_pipline(df, data_code)
pdata#['財報發布日']['2013':].sort_index(ascending=False)

{'季別': symbol     000930 000960 0009A0 1101 1102 1103 1104 1107 1108 1109  ... 9945  \
 datetime                                                            ...        
 2005-06-01    NaN    NaN    NaN    2    2    2    2    2    2    2  ...    2   
 2005-12-01    NaN    NaN    NaN    4    4    4    4    4    4    4  ...    4   
 2006-06-01    NaN    NaN    NaN    2    2    2    2    2    2    2  ...    2   
 2006-12-01    NaN    NaN    NaN    4    4    4    4    4    4    4  ...    4   
 2007-06-01    NaN    NaN    NaN    2    2    2    2    2    2    2  ...    2   
 ...           ...    ...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 2022-12-01    NaN    NaN    NaN    4    4    4    4  NaN    4    4  ...    4   
 2023-03-01    NaN    NaN    NaN    1    1    1    1  NaN    1    1  ...    1   
 2023-06-01      2      2      2    2    2    2    2  NaN    2    2  ...    2   
 2023-09-01      3      3      3    3    3    3    3  NaN    3    3  ...    3   
 2023-12-01      4    

In [5]:
pdata['財報發布日'].sort_index(ascending=False)

symbol,000930,000960,0009A0,1101,1102,1103,1104,1107,1108,1109,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-12-01,2024-03-11,2024-03-15,2024-03-14,2024-02-28,2024-03-15,2024-02-29,2024-03-15,NaT,2024-03-27,2024-03-13,...,2024-03-13,2024-02-23,2024-03-25,2024-03-15,2024-03-13,2024-03-14,NaT,2024-03-08,2024-03-14,2024-03-12
2023-09-01,2023-11-08,2023-11-10,2023-11-13,2023-11-10,2023-11-13,2023-11-10,2023-11-13,NaT,2023-11-13,2023-11-09,...,2023-11-13,2023-11-13,2023-11-09,2023-11-14,2023-11-06,2023-11-09,NaT,2023-11-13,2023-11-10,2023-11-13
2023-06-01,2023-08-24,2023-08-30,2023-08-30,2023-08-10,2023-08-11,2023-08-11,2023-08-11,NaT,2023-08-14,2023-08-10,...,2023-08-11,2023-08-14,2023-08-09,2023-08-14,2023-08-08,2023-08-09,2023-08-14,2023-08-11,2023-08-10,2023-08-09
2023-03-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2022-12-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2006-12-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2006-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [6]:
data_code = "TWN/EWIFINQ"
df_ = df.drop(columns='_id')
pdata = TWMarketMonitor().pdata_pipline(df_, data_code)

clean_pdata_dic = {}
for key in list(pdata.keys()):
    sorted_data = pdata[key].sort_index(ascending=False)
    clean_df = sorted_data.dropna(axis = 1, how = 'all')
    clean_pdata_dic[key] = clean_df
pmart = TWMarketMonitor().pmart_pipline(clean_pdata_dic, data_code, 90)
pmart 

ValueError: cannot reindex on an axis with duplicate labels

In [8]:
clean_pdata_dic["財報發布日"]

symbol,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,1110,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-12-01,2024-03-11,2024-03-15,2024-03-14,2024-02-28,2024-03-15,2024-02-29,2024-03-15,2024-03-27,2024-03-13,2024-03-15,...,2024-03-13,2024-02-23,2024-03-25,2024-03-15,2024-03-13,2024-03-14,NaT,2024-03-08,2024-03-14,2024-03-12
2023-09-01,2023-11-08,2023-11-10,2023-11-13,2023-11-10,2023-11-13,2023-11-10,2023-11-13,2023-11-13,2023-11-09,2023-11-10,...,2023-11-13,2023-11-13,2023-11-09,2023-11-14,2023-11-06,2023-11-09,NaT,2023-11-13,2023-11-10,2023-11-13
2023-06-01,2023-08-24,2023-08-30,2023-08-30,2023-08-10,2023-08-11,2023-08-11,2023-08-11,2023-08-14,2023-08-10,2023-08-11,...,2023-08-11,2023-08-14,2023-08-09,2023-08-14,2023-08-08,2023-08-09,2023-08-14,2023-08-11,2023-08-10,2023-08-09
2023-03-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2022-12-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2006-12-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2006-06-01,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [7]:
clean_pdata_dic['每股盈餘']

date_df_code = "TWN/EWIPRCD"
date_df = tejapi.get(date_df_code, chinese_column_name=True,paginate=True)
for_date_dict = TEJHandler().pdata_pipline(date_df, date_df_code)


releaserankpct = source_dict[release].rank(axis=1,pct=True)
filter = source_dict[column][releaserankpct < signal_delay]
filter.index = source_dict[release][releaserankpct < signal_delay].max(axis=1)
data_reindex = filter.reindex(for_date_dict['指數收盤價'].index, method = 'ffill')
factor_dict[column] = data_reindex 


symbol,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,1110,...,9945,9946,9949,9950,9951,9955,9957,9958,9960,9962
datetime,,,,,,,,,,,,,,,,,,,,,
2023-12-01,2.04,3.84,1.92,1.00,1.60,-0.32,2.72,0.56,2.60,-0.04,...,-4.40,0.92,-0.2,-0.60,5.84,-1.08,NaN,7.80,2.36,0.56
2023-09-01,2.96,5.04,3.28,0.68,3.28,0.72,5.68,1.88,1.36,0.40,...,8.40,-0.24,-0.8,1.92,6.08,-0.80,NaN,5.28,3.88,1.28
2023-06-01,2.88,4.44,2.28,1.80,5.04,0.00,2.48,1.48,2.40,0.48,...,6.08,0.32,-0.8,22.32,5.76,-0.08,-1.88,3.96,1.48,1.32
2023-03-01,NaN,NaN,NaN,0.20,0.81,0.03,0.42,0.31,0.42,0.06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,0.36,0.18,0.05,0.93,0.68,0.34,0.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-06-01,NaN,NaN,NaN,1.17,2.01,0.36,0.14,0.15,0.32,0.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-01,NaN,NaN,NaN,1.42,1.39,0.51,0.43,0.13,0.37,-0.16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-06-01,NaN,NaN,NaN,0.81,1.57,4.85,0.82,0.54,1.13,0.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
keys_list = list(pdata.keys())
keys_list

['季別',
 '合併(Y/N)',
 '單季(Q)/單半年(H)',
 '月份',
 '幣別',
 '財報發布日',
 '每股盈餘',
 'ROA(C) 稅前息前折舊前',
 'ROE(A)-稅後',
 '營業毛利率',
 '營業利益率',
 '稅後淨利率',
 'CFO/負債',
 'CFO/合併總損益',
 '現金流量比率',
 '每股淨值(F)-TSE公告數',
 '營收成長率',
 '營業毛利成長率',
 '營業利益成長率',
 '流動比率',
 '速動比率',
 '負債比率',
 '長期資金適合率(A)',
 '應收帳款週轉次數',
 '總資產週轉次數',
 '存貨週轉率(次)',
 '應收帳款及票據',
 '存貨',
 '流動資產',
 '採權益法之長期股權投資',
 '不動產廠房及設備',
 '商譽及無形資產合計',
 '非流動資產',
 '資產總額',
 '短期借款',
 '應付商業本票∕承兌匯票',
 '一年內到期長期負債',
 '流動負債',
 '應付公司債－非流動',
 '銀行借款－非流動',
 '其他長期借款－非流動',
 '非流動負債',
 '負債總額',
 '股本',
 '保留盈餘',
 '股東權益總額',
 '負債及股東權益總額',
 '營業收入淨額',
 '營業成本',
 '營業毛利',
 '營業費用',
 '營業利益',
 '營業外收入及支出',
 '所得稅費用',
 '合併總損益',
 '歸屬母公司淨利（損）',
 '折舊－CFO',
 '攤提－CFO',
 '來自營運之現金流量',
 '投資活動之現金流量',
 '籌資活動之現金流量']

In [14]:
pdata.keys().to_list()

AttributeError: 'dict_keys' object has no attribute 'to_list'

In [2]:
get_historical_returns = TWMarketMonitor().get_market_returns(review_month = 1)